# Influence functions for data mis-labelling

Both data mis-labelling and outlier detection target the same problem, they are operating on discrete and continuous values, respectively. Imagine a simple classification problem, where $y_i \in \{1, \dots, K\}$, the goal is now to find labels which are mislabelled. In our case we further simplify the problem to set $K=2$.

## 1. Using a GMM to generate artificial data with unique decision boundary

A Gaussian mixture model (GMM) is used

$$
y_i \sim \text{Cat}\left ( K, p=\frac{1}{K} \right) \\
x_i \sim \mathcal{N}\left (\cdot |\mu_{y_i}, \sigma^2 I \right),
$$


In [1]:
import numpy as np

sigma = 0.2
mus = np.asarray([
    [0.0, 0.0],
    [1.0, 1.0]
])


num_classes = len(mus)
num_samples = 10000
num_features = 2



for generating the data with fixed means $\mu_1, \dots, \mu_k$ and standard deviation $\sigma$. Sampling is fairly easy and can be done by sampling $N$ target data points from a categorical distribution. Afterwards the features can be obtained by using the previously sampled target labels.

In [2]:
from valuation.utils import Dataset
from sklearn.model_selection import train_test_split

gaussian_cov = sigma * np.eye(num_features)
gaussian_chol = np.linalg.cholesky(gaussian_cov)
y = np.random.randint(num_classes, size=num_samples)
x = np.einsum('ij,kj->ki', gaussian_chol, np.random.normal(size=[num_samples, num_features])) + mus[y]

Using this model has the advantage, that the unique decision boundary is easy inferrable,

$$
\begin{align*}
\| x - \mu_1 \|^2 &= \| x - \mu_2 \|^2 \\
\| \mu_1 \|^2 -2 x^\mathsf{T} \mu_1 &= \| \mu_2 \|^2 -2 x^\mathsf{T} \mu_2 \\
\implies 0 &= 2 (\mu_2 - \mu_1)^\mathsf{T} x + \| \mu_1 \|^2 - \| \mu_2 \|^2 \\
0 &= \mu_1^\mathsf{T}x - \mu_2^\mathsf{T}x - \frac{1}{2} \mu_1^\mathsf{T} \mu_1 + \frac{1}{2} \mu_2^\mathsf{T} \mu_2
\end{align*}
$$

by using Bayesian decision theory. Further enforcing a functional form $f(z) = x = a z + b$ with $z \in \mathbf{R}$ onto $x \in \mathbf{R}^d$ yields the implicit decision function in terms of

$$
\begin{align*}
0 &= (\mu_2 - \mu_1)^\mathsf{T} (at + b) + \frac{1}{2} \| \mu_1 \|^2 - \| \mu_2 \|^2 \\
\implies f(t) &= \underbrace{\begin{bmatrix} 0 & 1 \\ -1 & 0 \end{bmatrix} (\mu_2 - \mu_1)}_a t + \underbrace{\frac{\mu_1 + \mu_2}{2}}_b
\end{align*}
$$

with vectors $a, b \in \mathbf{R}^2$. Now it is possible to obtain

In [ ]:
x_min = np.asarray([-2, -2])
x_max = np.asarray([3, 3])
z_linspace = np.linspace(-1.5, 1.5, 100).reshape([-1, 1])

a = np.asarray([[0, 1], [-1, 0]]) @ (mus[1] - mus[0])
b = np.sum(mus, axis=0) / 2
a = a.reshape([1, -1])
decision_boundary = z_linspace * a + b

 the decision boundary. The next step is to wrap the previously generated data into a dataset with separate training and test set.

In [ ]:
arg_flipper = lambda x1, x2, y1, y2: (x1, y1, x2, y2)
dataset = Dataset(*arg_flipper(*train_test_split(x, y, train_size=0.70)))

It is always a good idea to visualize the dataset. In the 2-dimensional case it is rather straight forward. Each class is represented by a unique color.

In [ ]:
import matplotlib.pyplot as plt

datasets = {
    'train': (dataset.x_train, dataset.y_train),
    'test': (dataset.x_test, dataset.y_test)
}
num_datasets = len(datasets)
fig, ax = plt.subplots(1, num_datasets, figsize=(12, 4))

for i, dataset_name in enumerate(datasets.keys()):
    x, y = datasets[dataset_name]
    ax[i].set_title(dataset_name)
    ax[i].set_xlim(x_min[0], x_max[0])
    ax[i].set_ylim(x_min[1], x_max[1])
    ax[i].plot(decision_boundary[:, 0], decision_boundary[:, 1], color="black")

    for v in np.unique(y):
        idx = np.argwhere(y == v)
        ax[i].scatter(x[idx, 0], x[idx, 1], label=str(v))


plt.legend()
plt.show()

Note that both the train and test set are plotted side by side as well as the previously determined decision boundary. The next section cares about how to calculate influences for this dataset under the assumption of using a logistic regression model for inferring the right labels.

## 2. Calculating influences using a (differentiable) logistic regression model

Using the pyDVL valuation library a model can be formalized and fitted by using just a few lines of code.

In [ ]:
from valuation.models.pytorch_model import PyTorchSupervisedModel, PyTorchOptimizer
from valuation.models.binary_logistic_regression import BinaryLogisticRegressionTorchModel
import torch.nn.functional as F

model = PyTorchSupervisedModel(
    model=BinaryLogisticRegressionTorchModel(num_features),
    objective=F.binary_cross_entropy,
    num_epochs=100,
    batch_size=128,
    optimizer=PyTorchOptimizer.ADAM_W,
    optimizer_kwargs={
        "lr": 0.005,
        "weight_decay": 0.005
    },
)
model.fit(
    dataset.x_train,
    dataset.y_train
)

Note how the objective is specified in the model class, because the Hessian and scores calculated throughout are with respect to this loss function. 

Next the influences with respect to the previously fitted logistic regression model are calculated. A influence function $I(x_1, x_2) \colon \mathbb{R}^n \times \mathbb{R}^n \to \mathbb{R} $ measures the influence of the data point $x_1$ onto $x_2$ conditioned on the training targets $y_1$ and $y_2$. As long as the loss function $L(x, y)$ is differentiable (or can be approximated by a surrogate objective).

In [ ]:
from valuation.influence.general import influences
from valuation.influence.types import InfluenceTypes
train_influences = influences(
    model,
    dataset.x_train,
    dataset.y_train,
    dataset.x_test,
    dataset.y_test,
    influence_type=InfluenceTypes.Up
)
test_influences = influences(
    model,
    dataset.x_test,
    dataset.y_test,
    influence_type=InfluenceTypes.Up
)

Afterwards the average absolute influence (MAI) is calculated, the formula is given by
$$\text{MAI}(x) = \frac{1}{N} \sum_{i=1}^N | I(x, x_i) |.$$

In [ ]:
mean_influences = lambda arr: np.mean(np.abs(arr), axis=0)
mean_train_influences = mean_influences(train_influences)
mean_test_influences = mean_influences(test_influences)

And afterwards visualized by

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()

fig, ax = plt.subplots(1, num_datasets, figsize=(12, 4))
mean_influences = {
    'train': mean_train_influences,
    'test': mean_test_influences
}
v_max = max(np.max(mean_train_influences), np.max(mean_test_influences))
for i, dataset_name in enumerate(datasets.keys()):
    x = datasets[dataset_name][0].copy()
    ax[i].set_title(dataset_name)
    ax[i].set_xlim(x_min[0], x_max[0])
    ax[i].set_ylim(x_min[1], x_max[1])
    ax[i].plot(decision_boundary[:, 0], decision_boundary[:, 1], color="black")
    points = ax[i].scatter(x[:, 0], x[:, 1], c=mean_influences[dataset_name], vmin=0, vmax=v_max, cmap="plasma")

plt.suptitle("Influences of training and test set.")
plt.colorbar(points)
plt.show()

## 3. Flipping 5% of the labels of the training set and identify them

It is assumed that our reference test set is not flipped and was checked. As the test set is much smalled then the train set, this is a viable solution. In comparison to the correct test set, 5% of the training set get flipped at random positions. Next it is shown how to identify flipped examples

In [ ]:
from copy import copy

flip_percentage = 0.05
flipped_dataset = copy(dataset)
flip_num_samples = int(flip_percentage * len(dataset.x_train))
idx = np.random.choice(len(dataset.x_train), replace=False, size=flip_num_samples)
flipped_dataset.y_train[idx] = 1 - flipped_dataset.y_train[idx]

Start by fitting a new model

In [ ]:
from valuation.models.pytorch_model import PyTorchSupervisedModel, PyTorchOptimizer
from valuation.models.binary_logistic_regression import BinaryLogisticRegressionTorchModel
import torch.nn.functional as F

flipped_model = PyTorchSupervisedModel(
    model=BinaryLogisticRegressionTorchModel(num_features),
    objective=F.binary_cross_entropy,
    num_epochs=100,
    batch_size=128,
    optimizer=PyTorchOptimizer.ADAM_W,
    optimizer_kwargs={
        "lr": 0.005,
        "weight_decay": 0.005
    },
)
flipped_model.fit(
    flipped_dataset.x_train,
    flipped_dataset.y_train
)

to the flipped dataset and use it to calculate the influences and derive the absolute mean influences for all samples again.

In [ ]:
flipped_train_test_influences = influences(
    flipped_model,
    flipped_dataset.x_train,
    flipped_dataset.y_train,
    flipped_dataset.x_test,
    flipped_dataset.y_test,
    influence_type=InfluenceTypes.Up
)
mean_flipped_train_test_influences = np.mean(np.abs(flipped_train_test_influences), axis=0)

To make an comparison how good it approximates the randomly flipped samples, we take the 5% training samples with the highest absolute mean influence and measure the accuracy.

In [ ]:
estimated_idx = np.flip(np.argsort(mean_flipped_train_test_influences))[:len(idx)]
found_elements = set(estimated_idx).intersection(set(idx))
remaining_element = set(idx).difference(set(estimated_idx))
f"Around {100* len(found_elements) / len(idx):.2f}% could be identified. But there are {100* len(remaining_element) / len(idx):.2f}% remaining samples"

Depending on the sampled dataset a detection of up to 76.43% percent of the training data could be achieved. One might further inspect the selection method for the indices as it only selects the 200 highest influence points.

Depending on the sampled dataset a detection of up to 76.43% percent of the training data could be achieved. One might further inspect the selection method for the indices as it only selects the 200 highest influence points.